In [1]:
%matplotlib qt
import src
from src.simul_sweep import SimulSweep
from src.daq_driver import Daq
from src.base_sweep import BaseSweep
from src.sweep0d import Sweep0D
from src.sweep1d import Sweep1D
from src.sweep2d import Sweep2D
from src.sweep_ips import SweepIPS
from src.sweep_queue import SweepQueue, DatabaseEntry
from src.util import init_database
from src.tracking import *
import nidaqmx
import time, os
import numpy as np
import qcodes as qc
from src.util import connect_to_station, connect_station_instruments
from qcodes import initialise_or_create_database_at, load_by_run_spec
from qcodes import Station
from qcodes.instrument_drivers.Lakeshore.Model_372 import Model_372
from qcodes.instrument_drivers.american_magnetics.AMI430 import AMI430
from qcodes.instrument_drivers.stanford_research.SR860 import SR860
from qcodes_contrib_drivers.drivers.Oxford.IPS120 import OxfordInstruments_IPS120
from qcodes.dataset.measurements import Measurement
from qcodes.dataset.data_set import load_by_run_spec
from qcodes.dataset.data_export import get_data_by_id
from qcodes.instrument_drivers.stanford_research.SR830 import SR830

Logging hadn't been started.
Activating auto-logging. Current session state plus future input saved.
Filename       : C:\Users\nanouser\.qcodes\logs\command_history.log
Mode           : append
Output logging : True
Raw input log  : False
Timestamping   : True
State          : active
Qcodes Logfile : C:\Users\nanouser\.qcodes\logs\210611-10372-qcodes.log


In [2]:
station =Station(config_file='cfg/default.station.yaml')

In [3]:
station.snapshot()

{'instruments': {},
 'parameters': {},
 'components': {},
 'config': {'instruments': ordereddict([('daq', ordereddict([('address', 'Dev1'), ('enable_forced_reconnect', True), ('type', 'src.daq_driver.Daq')])), ('itc', ordereddict([('address', 'GPIB0::24::INSTR'), ('enable_forced_reconnect', True), ('type', 'src.ITC503.ITC503')])), ('keithley', ordereddict([('address', 'USB0::0x05E6::0x2450::04358606::INSTR'), ('enable_forced_reconnect', True), ('type', 'qcodes.instrument_drivers.tektronix.Keithley_2450.Keithley2450')])), ('lockin', ordereddict([('address', 'USB0::0xB506::0x2000::003594::INSTR'), ('enable_forced_reconnect', True), ('type', 'qcodes.instrument_drivers.stanford_research.SR860.SR860')])), ('ls340', ordereddict([('address', 'GPIB0::12::INSTR'), ('enable_forced_reconnect', True), ('type', 'src.Model_340.Model_340')]))])}}

In [4]:
station.snapshot()['config']['instruments'].items()

In [4]:
instruments = {}
for (name, other) in station.snapshot()['config']['instruments'].items():
    instruments[name] = station.load_instrument(name)

Connected to: NI DAQ USB-6259 (BNC) (Dev1) in 1.19s
Connected to: OXFORD ITC503 (serial:None, firmware:1.11) in 0.75s


2021-06-11 11:16:23,627 ¦ qcodes.instrument.base ¦ WARNING ¦ base ¦ snapshot_base ¦ 215 ¦ [keithley_source(Source2450)] Snapshot: Could not update parameter: sweep_axis


Connected to: KEITHLEY INSTRUMENTS 2450 (serial:04358606, firmware:1.6.4c) in 0.08s


2021-06-11 11:16:28,636 ¦ qcodes.instrument.base ¦ WARNING ¦ base ¦ snapshot_base ¦ 215 ¦ [keithley_source(Source2450)] Snapshot: Could not update parameter: user_delay
2021-06-11 11:16:28,654 ¦ qcodes.instrument.base ¦ WARNING ¦ base ¦ snapshot_base ¦ 215 ¦ [keithley_source(Source2450)] Snapshot: Could not update parameter: sweep_axis
2021-06-11 11:16:33,667 ¦ qcodes.instrument.base ¦ WARNING ¦ base ¦ snapshot_base ¦ 215 ¦ [keithley_source(Source2450)] Snapshot: Could not update parameter: user_delay
2021-06-11 11:16:38,698 ¦ qcodes.instrument.base ¦ WARNING ¦ base ¦ snapshot_base ¦ 215 ¦ [keithley_sense(Sense2450)] Snapshot: Could not update parameter: user_delay
2021-06-11 11:16:43,715 ¦ qcodes.instrument.base ¦ WARNING ¦ base ¦ snapshot_base ¦ 215 ¦ [keithley_sense(Sense2450)] Snapshot: Could not update parameter: user_delay
2021-06-11 11:16:48,744 ¦ qcodes.instrument.base ¦ WARNING ¦ base ¦ snapshot_base ¦ 215 ¦ [keithley_sense(Sense2450)] Snapshot: Could not update parameter: use

Connected to: Stanford_Research_Systems SR860 (serial:003594, firmware:V1.51) in 0.08s
Connected to: LSCI 340 (serial:000000, firmware:021901) in 0.05s


In [7]:
print(instruments)
daq = instruments['daq']
keith = instruments['keithley']
ls340 = instruments['ls340']
itc = instruments['itc']
sr860 = instruments['lockin']

{'daq': <Daq: daq>, 'itc': <ITC503: itc>, 'keithley': <Keithley2450: keithley>, 'lockin': <SR860: lockin>, 'ls340': <Model_340: ls340>}


In [8]:
sr860.X.label = 'I_sd'
sr860.P.label = 'P_I_sd'
keith.source.voltage.label = 'V_bg'
daq.ao1.voltage.label = 'V_tg'

In [11]:
db = os.environ['MeasureItHome']+'\\Databases\\SW2.db'

## Initialize the magnet driver

In [9]:
magnet = OxfordInstruments_IPS120('magnet', 'GPIB0::25::INSTR', use_gpib=True)

## Let's make sure we're connected and everything is as we expect

In [10]:
magnet.identify()

'IPS120-10  Version 3.07  (c) OXFORD 1996'

In [11]:
magnet.examine()

System Status: 
Normal
Activity: 
Hold
Local/Remote status: 
Local and locked
Switch heater: 
Off magnet at zero (switch closed)
Mode: 
Tesla, Magnet sweep: fast
Polarity: 
Desired: Positive, Magnet: Positive, Commanded: Positive, Both contactors open


## Set the control status 
#### 0 : Local and locked
#### 1 : Remote and locked
#### 2 : Local and unlocked
#### 3 : Remote and unlocked

In [12]:
magnet.remote_status(3)

### We can check and set the field setpoints and the sweeprate of the field

In [ ]:
magnet.field_setpoint()

In [ ]:
magnet.field_setpoint(0)

In [ ]:
magnet.sweeprate_field()

In [ ]:
magnet.sweeprate_field(0.3)

## Check (and set) the activity status
#### 0 : Hold
#### 1 : Goto zero
#### 2 : Goto setpoint

In [25]:
magnet.activity(2)

## Turn the switch heater on to enable the field to be ramped

In [14]:
magnet.heater_on()

Setting switch heater... (wait 40s)


## Now, we can run to the field, either by calling 'run_to_field', or setting the activity to 'goto setpoint' 

In [15]:
magnet.run_to_field(0.1)

## If we want to stay at field to do measurements, hold the field in place by setting persistent mode

In [33]:
magnet.set_persistent()

Setting switch heater... (wait 40s)
Waiting for the switch heater to become superconducting


#### You can check that we're in persistent mode by checking the persistent field (field in the magnet) and the field (which is the current going through the leads- should be 0)

In [17]:
magnet.persistent_field()

0.1

In [18]:
magnet.field()

0.0

## Once done, leave persistent mode so we can ramp the field back to zero

In [93]:
magnet.leave_persistent_mode()

Setting switch heater... (wait 40s)


## Once we are done, we can go back to 0 either by 'run_to_field' or setting activity to 'goto zero'

In [99]:
magnet.run_to_field(0)

Done ramping voltage to -7
Sweeping voltage to 7 V
Starting experimental run with id: 154. 


## Finally, switch the heater off and set us to 'hold'

In [108]:
magnet.heater_off()
magnet.activity(0)

Setting switch heater... (wait 40s)


## Double check to make sure the field and current are 0, and it is set to hold, and you can turn the IPS power off

In [ ]:
keithley_tg.output_enabled(1)
keithley_bg.output_enabled(1)
keithley_sg.output_enabled(1)

In [115]:
try:
    # Make sure database_name and the path are set to the correct values!
    db = 'Databases\\wte2-graphene-5.7.2021.db'
    exp_name = "back-gate-leakage"
    sample_name = "B=0T_Vtg=-9V_T=4K"
    initialise_or_create_database_at(db)
    qc.new_experiment(name=exp_name, sample_name=sample_name)
except:
    print("Error opening database")

In [99]:
sweep = SweepIPS(magnet, 1, inter_delay=0.25, save_data=True, persistent_magnet=False, plot_data = True)
sweep.follow_param(sr860.X, sr860.P, keith.sense.current, daq.ai0.voltage, daq.ai1.voltage, daq.ai2.voltage, daq.ai3.voltage, daq.ai4.voltage, daq.ai5.voltage)

In [100]:
sweep.start()

Starting experimental run with id: 81. 
Checking the status of the magnet and switch heater.
Heater is already on, so the magnet was not in persistent mode
Done with the sweep, t=1201.780999999959 s
Done with the sweep!


In [ ]:
sweep.plotter.data_queue

In [34]:
sweep.stop()

NameError: name 'sweep' is not defined

In [43]:
sweep.kill()

In [152]:
sweep_gate = Sweep1D(keith.source.voltage, 0, -20, .1, inter_delay=0.2, continual=False, bidirectional=False, save_data=False, plot_data=True)
sweep_gate.follow_param(keith.sense.current)

In [125]:
keith.source.voltage()

-6.95

In [1]:
daq.ao1.voltage()


NameError: name 'daq' is not defined

In [153]:
sweep_gate.start()

Ramping to our starting setpoint value of 0 V
Sweeping voltage to 0 V
Ramping voltage to 0 . . . 
Done ramping voltage to 0
Sweeping voltage to -20 V


In [104]:
sweep_gate.stop()

Sweep not currently running. Nothing to stop.


In [105]:
sweep_gate.kill()

In [94]:
daq.ai6.voltage()

1.7027949103388373

In [ ]:
ramp_sg = Sweep1D(keithley_sg.source.voltage, 0, 10, -.1, inter_delay=0.25, save_data=False, x_axis_time = False)
ramp_sg.follow_param(daq.ai0.voltage, daq.ai1.voltage, daq.ai2.voltage, daq.ai3.voltage, daq.ai4.voltage, daq.ai5.voltage, sr830.X, sr830.P, sr860.X, sr860.P)

In [ ]:
ramp_sg.start()

In [6]:
sweep_queue = SweepQueue(inter_delay = 1)
exp_name = '210203-Landau_fan_unknown_temp_Vsg=10V'

Vtg_min, Vtg_max = -8., 8.
Vtg_step_slow = 0.05
Vtg_step_fast = 0.2

Vbg_min, Vbg_max = -13., 13.
Vbg_step_slow = 0.08
Vbg_step_fast = 0.3

inter_delay_slow = 0.5
inter_delay_fast = 0.25

N = 60 #data points
B_start, B_end = 2.5, 6.25
B_crit = 5

for B in np.linspace(B_start, B_end, N):
    #Magnet sweep
    if B>=B_crit:
        persistent_magnet = True
    else:
        persistent_magnet = False
    mag_sweep = SweepIPS(magnet, B, inter_delay=0.25, save_data=False, persistent_magnet=persistent_magnet)
    #mag_sweep.follow_param(ls340.A.temperature)
    
    # Create the database entry
    sample_name = f'B={B:.2f}T_sweep-mag'
    db_entry = DatabaseEntry(db, exp_name, sample_name)
    sweep_queue.append(db_entry)
    sweep_queue.append(mag_sweep)
    
    # TG sweep
    tg_initialize = Sweep1D(keithley_tg.source.voltage, 0, Vtg_min, Vtg_step_fast, inter_delay=inter_delay_fast, x_axis_time=0, save_data = False)
    tg_sweep = Sweep1D(keithley_tg.source.voltage, Vtg_min, Vtg_max, Vtg_step_slow, inter_delay=inter_delay_slow, x_axis_time=0, save_data = True)
    tg_rampdown = Sweep1D(keithley_tg.source.voltage, Vtg_max, 0, Vtg_step_fast, inter_delay=inter_delay_fast, x_axis_time=0, save_data = False)
    
    tg_sweep.follow_param(daq.ai0.voltage, daq.ai1.voltage, daq.ai2.voltage, daq.ai3.voltage, daq.ai4.voltage, daq.ai5.voltage, sr830.X, sr830.P, sr860.X, sr860.P)
    
    # Create the database entry
    sample_name = f'B={B:.2f}T_Vbg=0V_sweep-Vtg'
    db_entry = DatabaseEntry(db, exp_name, sample_name)
    sweep_queue.append(db_entry)
    sweep_queue.append(tg_initialize, tg_sweep, tg_rampdown)

    bg_initialize = Sweep1D(keithley_bg.source.voltage, 0, Vbg_min, Vbg_step_fast, inter_delay=inter_delay_fast, x_axis_time=0, save_data = False)
    bg_sweep = Sweep1D(keithley_bg.source.voltage, Vbg_min, Vbg_max, Vbg_step_slow, inter_delay=inter_delay_slow, x_axis_time=0, save_data = True)
    bg_rampdown = Sweep1D(keithley_bg.source.voltage, Vbg_max, 0, Vbg_step_fast, inter_delay=inter_delay_fast, x_axis_time=0, save_data = False)
    
    bg_sweep.follow_param(daq.ai0.voltage, daq.ai1.voltage, daq.ai2.voltage, daq.ai3.voltage, daq.ai4.voltage, daq.ai5.voltage, sr830.X, sr830.P, sr860.X, sr860.P)
    
    # Create the database entry
    sample_name = f'B={B:.2f}T_Vtg=0V_sweep-Vbg'
    db_entry = DatabaseEntry(db, exp_name, sample_name)
    sweep_queue.append(db_entry)
    sweep_queue.append(bg_initialize, bg_sweep, bg_rampdown)
    


NameError: name 'magnet' is not defined

In [26]:
sweep_queue = SweepQueue(inter_delay = 1)

Vtg_min = -3
Vtg_max = +3
Vtg_prev = -3

db = "Databases\\wte2-graphene-5.7.2021.db"
exp_name = "2D-gate-sweep-3-7_T=4K_G=1e6_ground-others"
#Previous value = 141
for Vtg in np.linspace(Vtg_min, Vtg_max,141, endpoint=True):
    Vbg_min = round(-3 + Vtg, 2)
    Vbg_max = round(+3 + Vtg, 2)
    #print(Vtg, Vbg_min, Vbg_max)
    
    # Create the database entry
    sample_name = f'Vtg={Vtg:.2f}'
    db_entry = DatabaseEntry(db, exp_name, sample_name)
    
    tg_sweep = Sweep1D(daq.ao1.voltage, Vtg_prev, Vtg, .02, inter_delay=0.2, bidirectional=False, save_data=False, plot_data=True)
    tg_sweep.follow_param(sr860.X, sr860.P, keith.sense.current)
    
    bg_sweep = Sweep1D(keith.source.voltage, Vbg_min, Vbg_max, .02, inter_delay=0.2, bidirectional=True, save_data=True, plot_data=True)
    bg_sweep.follow_param(sr860.X, sr860.P, keith.sense.current)
    
    sweep_queue.append(db_entry, tg_sweep, bg_sweep)
    
    Vtg_prev = Vtg
    
# Append final sweeps back to 0V
tg_sweep_down = Sweep1D(daq.ao1.voltage, Vtg_max, 0, .1, inter_delay=0.2, bidirectional=False, save_data=False, plot_data=True)
bg_sweep_down = Sweep1D(keith.source.voltage, Vtg_max*2, 0, .1, inter_delay=0.2, bidirectional=False, save_data=False, plot_data=True)
sweep_queue.append(db_entry, tg_sweep_down, bg_sweep_down)

In [22]:
keith.source.voltage(0)

In [27]:
for sweep in sweep_queue.queue:
    print(sweep)

Database entry saving to Databases\wte2-graphene-5.7.2021.db with experiment name 2D-gate-sweep-3-7_T=4K_G=1e6_ground-others and sample name Vtg=-3.00.
1D Sweep of daq_ao1_voltage from -3 to -3.0, with step size -0.02.
1D Sweep of keithley_source_voltage from -6.0 to 0.0, with step size 0.02.
Database entry saving to Databases\wte2-graphene-5.7.2021.db with experiment name 2D-gate-sweep-3-7_T=4K_G=1e6_ground-others and sample name Vtg=-2.96.
1D Sweep of daq_ao1_voltage from -3.0 to -2.9571428571428573, with step size 0.02.
1D Sweep of keithley_source_voltage from -5.96 to 0.04, with step size 0.02.
Database entry saving to Databases\wte2-graphene-5.7.2021.db with experiment name 2D-gate-sweep-3-7_T=4K_G=1e6_ground-others and sample name Vtg=-2.91.
1D Sweep of daq_ao1_voltage from -2.9571428571428573 to -2.914285714285714, with step size 0.02.
1D Sweep of keithley_source_voltage from -5.91 to 0.09, with step size 0.02.
Database entry saving to Databases\wte2-graphene-5.7.2021.db with ex

In [16]:
keith.output_enabled(True)

In [28]:
sweep_queue.start()

Starting sweeps
Starting sweep of V_tg from -3                           V to -3.0 V
Ramping to our starting setpoint value of -3 V
Sweeping V_tg to -3 V
Ramping V_tg to -3 . . . 
Done ramping V_tg to -3
Sweeping V_tg to -3.0 V
Finished sweep of V_tg from -3.0                   V to -3 V
Starting sweep of V_bg from -6.0                           V to 0.0 V
Ramping to our starting setpoint value of -6.0 V
Sweeping V_bg to -6.0 V
Ramping V_bg to -6.0 . . . 
Done ramping V_bg to -6.0
Sweeping V_bg to 0.0 V
Starting experimental run with id: 439. 
Finished sweep of V_bg from -6.0                   V to 0.0 V
Starting sweep of V_tg from -3.0                           V to -2.9571428571428573 V
Ramping to our starting setpoint value of -3.0 V
Done ramping V_tg to -3.0
Sweeping V_tg to -2.9571428571428573 V
Finished sweep of V_tg from -2.9571428571428573                   V to -3.0 V
Starting sweep of V_bg from -5.96                           V to 0.04 V
Ramping to our starting setpoint value

In [19]:
sweep_queue.current_sweep.kill()

Sweep not currently running. Nothing to stop.


In [ ]:
print(sweep_queue.current_sweep)

In [ ]:
sweep.update_values()

In [18]:
sweep_queue.kill()

AttributeError: 'SweepQueue' object has no attribute 'kill'

In [ ]:
abs(self.setpoint-self.end)-abs(self.step/2)>=0

In [ ]:
sweep.save_data

In [51]:
abs(rs.end-(-1.7))-abs(rs.step/2)

8.673617379884035e-18

In [42]:
print(rs.begin, rs.end, rs.step)

-1.76 -1.71 0.02


In [37]:
rs = sweep_queue.current_sweep.ramp_sweep

In [40]:
print(rs.plotter.setaxline.get_ydata()[0:9])

[-1.76 -1.74 -1.72 -1.7  -1.68 -1.66 -1.64 -1.62 -1.6 ]


In [24]:
magnet.field_setpoint(0)

In [22]:
magnet.parameters

{'IDN': <qcodes.instrument.parameter.Parameter: IDN at 2628086942024>,
 'timeout': <qcodes.instrument.parameter.Parameter: timeout at 2628086943048>,
 'mode': <qcodes.instrument.parameter.Parameter: mode at 2628086941576>,
 'mode2': <qcodes.instrument.parameter.Parameter: mode2 at 2628086941192>,
 'activity': <qcodes.instrument.parameter.Parameter: activity at 2628086940360>,
 'switch_heater': <qcodes.instrument.parameter.Parameter: switch_heater at 2628086939912>,
 'field_setpoint': <qcodes.instrument.parameter.Parameter: field_setpoint at 2628086935432>,
 'sweeprate_field': <qcodes.instrument.parameter.Parameter: sweeprate_field at 2628086935240>,
 'system_status': <qcodes.instrument.parameter.Parameter: system_status at 2628086934088>,
 'system_status2': <qcodes.instrument.parameter.Parameter: system_status2 at 2628086934408>,
 'polarity': <qcodes.instrument.parameter.Parameter: polarity at 2628086932936>,
 'voltage': <qcodes.instrument.parameter.Parameter: voltage at 2628086933128>

In [70]:
daq.ai0.voltage() * (100e-3)/2.5

0.053280209960356775

In [60]:
daq.ai1.voltage()

6.198411302963402